In [ ]:
!pip install requests pandas

In [2]:
from skillExtracter import extractKeywordFromLightSkillAPI
import json
from util import getAccessToken
import pandas as pd

In [ ]:
df = pd.read_csv('data/job_descriptions/jobs.csv')

In [ ]:
df.columns

In [ ]:
token = json.loads(getAccessToken())

In [ ]:
type(1.1) == type(2.2)

In [ ]:
# row.description is pd.isnull or row.description == None or pd.isna(row.description)

In [ ]:
data = []

chunksize = 1000
for chunk in pd.read_csv('data/job_descriptions/jobs.csv', chunksize=chunksize):
    print("processing 1000 record")
    for row in chunk.itertuples(index=False):
        if pd.isna(row.description):
            continue
        job = {}
        job['description'] = row.description
        job['Industries'] = row.Industries
        job['company'] = row.company
        job['context'] = row.context
        job['education'] = row.education
        job['location'] = row.location
        job['months_experience'] = row.months_experience
        job['sal_high'] = row.sal_high
        job['sal_low'] = row.sal_low
        job['salary'] = row.salary
        job['title'] = row.title
        skills = extractKeywordFromLightSkillAPI(row.description, token)     
        job['skills'] = ",".join(skills)
        data.append(job)
    print('processed 1000 data')

newdf = pd.DataFrame(data)


In [ ]:
newdf.describe

In [ ]:
newdf.to_csv("jd_with_keyword.csv")

In [3]:
df = pd.read_csv('../data/job_descriptions/jd_with_keyword.csv')

In [7]:
df

,Unnamed: 0,description,Industries,company,context,education,location,months_experience,sal_high,sal_low,salary,title,skills
0,0,Job Title: Senior Data Engineer Location: Alex...,Broadcast Media,CyberCoders,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",bachelor degree,"Alexandria, VA",60.0,NaN,NaN,NaN,Senior Data Engineer,"NoSQL,Agile Methodology,Unit Testing,Data Proc..."
1,1,"Ethicon, part of Johnson & Johnson Medical Dev...","Hospital & Health Care, Medical Devices, and P...",Johnson & Johnson,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",bachelor degree,"Santa Clara, CA",96.0,NaN,NaN,NaN,Principal Full Stack Software Engineer.,"Angular (Web Framework),Software Configuration..."
2,2,Microsoft’s WCB health team is looking for a S...,"Computer Hardware, Computer Software, and Info...",Microsoft,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",bachelor degree,"Washington, DC",84.0,189000.0,120000.0,"$120,000.00/yr - $189,000.00/yr",Senior Software Engineer,"Microsoft Azure,Concision,Debugging,Database A..."
3,3,Microsoft’s WCB health team is looking for a S...,"Computer Hardware, Computer Software, and Info...",Microsoft,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",bachelor degree,"Reston, VA",84.0,189000.0,120000.0,"$120,000.00/yr - $189,000.00/yr",Senior Software Engineer,"Microsoft Azure,Concision,Debugging,Database A..."
4,4,Microsoft’s WCB health team is looking for a S...,"Computer Hardware, Computer Software, and Info...",Microsoft,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",bachelor degree,"Irving, TX",84.0,175000.0,108000.0,"$108,000.00/yr - $175,000.00/yr",Senior Software Engineer,"Microsoft Azure,Concision,Debugging,Database A..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7775,7775,Who We Are: The Oracle's State & Local Global ...,"Information Technology and Services, Computer ...",Oracle,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",bachelor degree,United States,60.0,NaN,NaN,NaN,Senior Site Reliability Engineer,"Kibana,C++ (Programming Language),Fault Tolera..."
7776,7776,Primary Skills: 1. Good hands on experience on...,NaN,Sky Solutions,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",bachelor degree,"Malvern, PA",NaN,NaN,NaN,NaN,Site Reliability Engineer,"Spring Boot,Github,Kubernetes,RESTful API,Apac..."
7777,7777,About The Team The SRE team at Zillow Group em...,"Marketing and Advertising, Computer Software, ...",Zillow,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",bachelor degree,United States,60.0,NaN,NaN,NaN,Senior Site Reliability Engineer,"Debugging,Software Systems,Real Estate Transac..."
7778,7778,PlayStation isn’t just the Best Place to Play ...,"Computer Software, Consumer Services, and Ente...",PlayStation,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",bachelor degree,"San Diego, CA",84.0,162000.0,129000.0,"$129,000.00/yr - $162,000.00/yr",Sr. Site Reliability Engineer,"Java Virtual Machine (JVM),MySQL,Cloud Infrast..."


In [ ]:
!pip install pinecone-client openai python-dotenv

In [3]:
import openai

import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [4]:
EMBEDDING_MODEL = "text-embedding-3-small"

In [5]:
from dotenv import load_dotenv

load_dotenv() 
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "None"))

In [ ]:
# # old code updated code is at below sale with method name createEmbeddingWithValues
# embeddings = []
# chunksize = 1000
# for chunk in pd.read_csv('jd_with_keyword.csv', chunksize=chunksize):
#     print("processing 1000 record")
#     for row in chunk.itertuples(index=False):
#         response = client.embeddings.create(model=EMBEDDING_MODEL, input=chunk)
#         for i, be in enumerate(response.data):
#             assert i == be.index
#         batch_embeddings = [e.embedding for e in response.data]
#         embeddings.extend(batch_embeddings)
#     print("processed 1000 record")

In [ ]:
df.columns

In [6]:
def createEmbeddingWithValues(data, extras):
    result, __data =[], {}
    response = client.embeddings.create(model=EMBEDDING_MODEL, input=data)
    for i, be in enumerate(response.data):
        assert i == be.index
    batch_embeddings = [e.embedding for e in response.data]
    __data["id"] = str(extras['id'])
    __data["values"] = batch_embeddings[0]
    __data["metadata"] = {'extras': str(extras)}
    result.append(__data)
    return result

In [7]:
def convertToDic(row):
    dic = {}
    dic["id"] = str(row._0)
    dic["description"] = str(row.description)
    dic["industries"] = "" if pd.isnull(row.Industries) else str(row.Industries)
    dic["company"] = "" if pd.isnull(row.company) else str(row.company)
    dic["education"] = "" if pd.isnull(row.education) else str(row.education)
    dic["months_experience"] = 0 if pd.isna(row.months_experience) else row.months_experience
    dic["sal_high"] = 0 if pd.isna(row.sal_high) else row.sal_high
    dic["sal_low"] = 0 if pd.isna(row.sal_low) else row.sal_low
    dic["salary"] = 0 if pd.isna(row.salary) else row.salary
    dic["title"] = "" if pd.isnull(row.title) else str(row.title)
    dic["skills"] = "" if pd.isnull(row.skills) else str(row.skills)
    return dic
    

In [8]:
description_embeddings = []
skills_embeddings = []
chunksize = 500
for chunk in pd.read_csv('../data/job_descriptions/jd_with_keyword.csv', chunksize=chunksize):
    print("processing 500")
    for row in chunk.itertuples(index=False):
        if pd.isna(row.description) or pd.isna(row.skills):
            continue
        description_embeddings.extend(createEmbeddingWithValues(row.description, convertToDic(row)))
        skills_embeddings.extend(createEmbeddingWithValues(row.skills, convertToDic(row)))
    print("processed 500")

description_embeddings_df = pd.DataFrame(description_embeddings)
skills_embeddings_df = pd.DataFrame(skills_embeddings)

processed 500
processing 500
processed 500
processing 500
processed 500
processing 500
processed 500
processing 500
processed 500
processing 500
processed 500
processing 500
processed 500
processing 500
processed 500


In [ ]:
description_embeddings_df.to_csv("../data/embeddings/description_embeddings.csv")

In [ ]:
skills_embeddings_df.to_csv("../data/embeddings/skills_embeddings.csv")

In [ ]:
description_embeddings_df = pd.read_csv("../data/embeddings/description_embeddings.csv")
skills_embeddings_df = pd.read_csv("../data/embeddings/skills_embeddings.csv")

In [9]:
from pinecone import Pinecone, ServerlessSpec
_api_key = os.getenv("PINECONE_API_KEY")
print(_api_key)
pc = Pinecone(
    api_key="432efbce-a192-4eb1-bba7-6b46fb18f1b5"
)

432efbce-a192-4eb1-bba7-6b46fb18f1b5


/Users/aniketgiram/Desktop/Big-Data/JobFitAI/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [10]:
job_fit_index_name = 'job-fit'

In [ ]:


# Check whether the index with the same name already exists - if so, delete it
if job_fit_index_name in pc.list_indexes():
    pc.delete_index(job_fit_index_name)

# Now do stuff
if job_fit_index_name not in pc.list_indexes().names():
    pc.create_index(
        name=job_fit_index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [11]:
index = pc.Index(job_fit_index_name)

In [ ]:
index.describe_index_stats()

In [ ]:
index.upsert(
  vectors=description_embeddings,
  namespace="description"
)

In [12]:
def chunker(seq, size):
    """Yields successive chunks from seq."""
    for pos in range(0, len(seq), size):
        yield seq.iloc[pos:pos + size]

In [13]:
def convert_data(chunk):
    """Converts a pandas DataFrame chunk to the format expected by Pinecone's upsert method."""
    data = []
    for _, row in chunk.iterrows():
        vector_id = str(row['id'])
        embedding = row['values']
        metadata = row.get('metadata', {})
        if 'context' in eval(metadata['extras']):
            extras = eval(metadata['extras'])
            del extras['context']
            metadata = extras
        data.append((vector_id, embedding, metadata))
    return data

In [ ]:
description_embeddings_df

In [ ]:
chunk_size = 100  # Define your chunk size
for chunk in chunker(description_embeddings_df, chunk_size):
    vectors_to_upsert = convert_data(chunk)
    print(vectors_to_upsert)
    print("vectorising chunk")
    index.upsert(
        vectors=vectors_to_upsert,
        namespace="description",
    )
    print("upserted chunk")
    

In [14]:
chunk_size = 200  # Define your chunk size
for chunk in chunker(skills_embeddings_df, chunk_size):
    vectors_to_upsert = convert_data(chunk)
    print("vectorising chunk")
    index.upsert(
        vectors=vectors_to_upsert,
        namespace="skills"
    )
    print("upserted chunk")

vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorisin

In [15]:
query = """
Job Title: Senior Data Engineer Location: Alexandria, VA Salary Range: $120k - $150k Requirements: ETL/ELT, SQL, AWS/Google Cloud, Linux/Unix, Spark (preferred), NoSQL (preferred), Machine Learning concepts (preferred) Based in beautiful Alexandria, VA, we are one of the hottest media analytics and software start-ups in the DC area. Due to growth, we are actively seeking to hire a Senior Data Engineer to join our team. The ideal candidate will have at least 5 years of experience with data pipelines (built with Python), ETL experience, a strong SQL background, experience with AWS or Google Cloud, and strong Unix/Linux fundamentals. Any experience with Spark, NoSQL, and Machine Learning would be a huge plus. If this sounds like you, please apply now or send your resume to shiv.warrier@cybercoders.com! What You Will Be Doing Lead design and development of data pipelines Deliver features on a cadence within an agile framework Contribute to the definition of user stories Collaborate with other members of the team, including offshore, on development integration Write unit tests and maintain high code quality, per both static code analysis team standards Be available on a rotating schedule for production issues What You Need for this Position Must-Have BS in Computer Science or equivalent 5+ years of experience in software/data engineering Expertise in ETL/ELT techniques Expertise in SQL Experience working with AWS or Google Cloud Strong Unix/Linux fundamentals Nice To Have Experience with data processing frameworks e.g. Spark Experience with NoSQL column-store databases Exposure to machine learning concepts Exposure to ad tech concepts terminology What's In It for You Competitive salary with high bonus potential Collaborative and creative atmosphere, with inspired leadership Career advancement opportunities Recognition and reward for outstanding performance Great medical, dental; vision Insurance packages Competitive 401K with company match to plan for the long term Unlimited paid-time-off Transportation benefits, cell phone reimbursement Casual dress all day, every day So, if you are a Senior Data Engineer with experience, please apply today! Email Your Resume In Word To Looking forward to receiving your resume through our website and going over the position with you. Clicking apply is the best way to apply, but you may also: Shiv.Warrier@CyberCoders.com Please do NOT change the email subject line in any way. You must keep the JobID: linkedin : SW3-1634173 -- in the email subject line for your application to be considered.*** Shiv Warrier - Sr. Executive Recruiter - CyberCoders Applicants must be authorized to work in the U.S. CyberCoders, Inc is proud to be an Equal Opportunity Employer All qualified applicants will receive consideration for employment without regard to race, color, religion, sex, national origin, disability, protected veteran status, or any other characteristic protected by law. Your Right to Work - In compliance with federal law, all persons hired will be required to verify identity and eligibility to work in the United States and to complete the required employment eligibility verification document form upon hire.
"""

xqq = client.embeddings.create(input=query, model=EMBEDDING_MODEL).data[0].embedding


In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'description': {'vector_count': 7768},
                'skills': {'vector_count': 7768}},
 'total_vector_count': 15536}

In [17]:
match_res = index.query(top_k=3, vector=xqq, namespace="description", include_metadata=True)

In [19]:
match_res

{'matches': [{'id': '1707',
              'metadata': {'extras': '{\'id\': \'1707\', \'description\': "Job '
                                     'Title: Senior Data Engineer Location: '
                                     'Alexandria, VA Salary Range: $120k - '
                                     '$150k Requirements: ETL/ELT, SQL, '
                                     'AWS/Google Cloud, Linux/Unix, Spark '
                                     '(preferred), NoSQL (preferred), Machine '
                                     'Learning concepts (preferred) Based in '
                                     'beautiful Alexandria, VA, we are one of '
                                     'the hottest media analytics and software '
                                     'start-ups in the DC area. Due to growth, '
                                     'we are actively seeking to hire a Senior '
                                     'Data Engineer to join our team. The '
                            

In [21]:
res = []
for i in match_res.matches:
    a = {}
    metadata = i['metadata']
    res.append(eval(metadata['extras'])['skills'].split(","))

In [22]:
res

[['NoSQL',
  'Agile Methodology',
  'Unit Testing',
  'Data Processing',
  'Unix',
  'Data Engineering',
  'Linux',
  'Python (Programming Language)',
  'Apache Spark',
  'Machine Learning',
  'Software Quality (SQA/SQC)',
  'SQL (Programming Language)',
  'Data Pipelines',
  'Amazon Web Services',
  'Static Program Analysis',
  'Extract Transform Load (ETL)',
  'Executive Recruitment',
  'Computer Science',
  'Google Cloud Platform (GCP)',
  'User Story',
  'Leadership'],
 ['NoSQL',
  'Agile Methodology',
  'Unit Testing',
  'Data Processing',
  'Unix',
  'Data Engineering',
  'Linux',
  'Python (Programming Language)',
  'Apache Spark',
  'Machine Learning',
  'Software Quality (SQA/SQC)',
  'SQL (Programming Language)',
  'Data Pipelines',
  'Amazon Web Services',
  'Static Program Analysis',
  'Extract Transform Load (ETL)',
  'Executive Recruitment',
  'Computer Science',
  'Google Cloud Platform (GCP)',
  'User Story',
  'Leadership'],
 ['NoSQL',
  'Agile Methodology',
  'Unit Te

In [23]:
common_skills = set(res[0])

# Iterate over the rest of the lists and find common elements
for skills in res[1:]:
    common_skills = common_skills.intersection(set(skills))

print(common_skills)

{'Extract Transform Load (ETL)', 'Executive Recruitment', 'Google Cloud Platform (GCP)', 'Data Processing', 'Unix', 'NoSQL', 'Leadership', 'Machine Learning', 'Apache Spark', 'Linux', 'Computer Science', 'SQL (Programming Language)', 'Amazon Web Services', 'Python (Programming Language)', 'Data Engineering', 'Static Program Analysis', 'Software Quality (SQA/SQC)', 'Agile Methodology', 'Unit Testing', 'User Story', 'Data Pipelines'}


In [24]:
common_skills = set(res[0])

# Iterate over the rest of the lists and find common elements
for skills in res[1:]:
    common_skills = common_skills.union(set(skills))

print(common_skills)

{'Extract Transform Load (ETL)', 'Executive Recruitment', 'Google Cloud Platform (GCP)', 'Data Processing', 'Unix', 'NoSQL', 'Leadership', 'Machine Learning', 'Apache Spark', 'Linux', 'Computer Science', 'SQL (Programming Language)', 'Amazon Web Services', 'Python (Programming Language)', 'Data Engineering', 'Static Program Analysis', 'Software Quality (SQA/SQC)', 'Agile Methodology', 'Unit Testing', 'User Story', 'Data Pipelines'}
